# Predict Winner from 0-25min Realtime Stats using RNN

In [1]:
import math
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.autograd import Variable
from tqdm import tqdm
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

RANDOM_SEED = 0
MAX_TIME_STEP = 30

## Dataset

In [2]:
df = pd.read_csv('../data/LeagueofLegends.csv', sep=',')
df = df[df['gamelength'] >= MAX_TIME_STEP]
df.reset_index(drop = True, inplace = True)
matches = len(df)
print(f'# of matches: {matches}')

from ast import literal_eval
df['golddiff'] = df['golddiff'].apply(literal_eval)
df[['golddiff']].head()

# of matches: 6384


,golddiff
0,"[0, 0, -14, -65, -268, -431, -488, -789, -494,..."
1,"[0, 0, -26, -18, 147, 237, -152, 18, 88, -242,..."
2,"[0, 0, 10, -60, 34, 37, 589, 1064, 1258, 913, ..."
3,"[0, 0, -15, 25, 228, -6, -243, 175, -346, 16, ..."
4,"[40, 40, 44, -36, 113, 158, -121, -191, 23, 20..."


In [3]:
def count_item(items):
    count = np.zeros(MAX_TIME_STEP, dtype=np.int8)
    for timestep in range(MAX_TIME_STEP) :
        for item in items:
            if item[0] <= timestep + 1:
                count[timestep] += 1
    return count

df['bDragons'] = df['bDragons'].apply(literal_eval)
df['rDragons'] = df['rDragons'].apply(literal_eval)

df['bDragons'] = df['bDragons'].apply(count_item)
df['rDragons'] = df['rDragons'].apply(count_item)
df['dragondiff'] = df['bDragons'] - df['rDragons']

df[['dragondiff']].tail()

,dragondiff
6379,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1,..."
6380,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1,..."
6381,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, ..."
6382,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ..."
6383,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [4]:
df['bBarons'] = df['bBarons'].apply(literal_eval)
df['rBarons'] = df['rBarons'].apply(literal_eval)

df['bBarons'] = df['bBarons'].apply(count_item)
df['rBarons'] = df['rBarons'].apply(count_item)
df['barondiff'] = df['bBarons'] - df['rBarons']

df[['barondiff']].head()

,barondiff
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [5]:
df['bHeralds'] = df['bHeralds'].apply(literal_eval)
df['rHeralds'] = df['rHeralds'].apply(literal_eval)

df['bHeralds'] = df['bHeralds'].apply(count_item)
df['rHeralds'] = df['rHeralds'].apply(count_item)
df['heralddiff'] = df['bHeralds'] - df['rHeralds']

df[['heralddiff']].head()

,heralddiff
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [6]:
df['bTowers'] = df['bTowers'].apply(literal_eval)
df['rTowers'] = df['rTowers'].apply(literal_eval)

df['bTowers'] = df['bTowers'].apply(count_item)
df['rTowers'] = df['rTowers'].apply(count_item)
df['towerdiff'] = df['bTowers'] - df['rTowers']

df[['towerdiff']].head()

,towerdiff
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -2, -2, ..."


In [7]:
df['bInhibs'] = df['bInhibs'].apply(literal_eval)
df['rInhibs'] = df['rInhibs'].apply(literal_eval)

df['bInhibs'] = df['bInhibs'].apply(count_item)
df['rInhibs'] = df['rInhibs'].apply(count_item)
df['inhibitordiff'] = df['bInhibs'] - df['rInhibs']

df[['inhibitordiff']].head()

,inhibitordiff
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [8]:
df['bKills'] = df['bKills'].apply(literal_eval)
df['rKills'] = df['rKills'].apply(literal_eval)

df['bKills'] = df['bKills'].apply(count_item)
df['rKills'] = df['rKills'].apply(count_item)
df['killdiff'] = df['bKills'] - df['rKills']

df[['killdiff']].head()

,killdiff
0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, ..."
2,"[0, 0, 0, 0, 0, 1, 2, 2, 1, 1, 2, 2, 5, 5, 5, ..."
3,"[0, 0, 0, 0, 0, 0, 0, -1, 0, -1, -1, -1, -1, -..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [9]:
stats = ['golddiff','dragondiff', 'barondiff', 'heralddiff', 'towerdiff', 'inhibitordiff', 'killdiff']
x = df[stats]
y = df['bResult']

x.tail()

,golddiff,dragondiff,barondiff,heralddiff,towerdiff,inhibitordiff,killdiff
6379,"[0, -8, -187, -37, -92, -164, -229, -424, -256...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -3, -3, -..."
6380,"[0, 0, -18, -95, 45, -87, -117, 199, 126, 92, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, ..."
6381,"[0, 0, -86, -39, -207, -349, -60, -140, 187, -...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -2,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, -1, -1, -1, -1, -1, -2, -2, -2, -4, ..."
6382,"[0, 0, -97, 33, 351, 284, 299, 263, 403, 623, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6383,"[0, 0, -8, -225, -36, 73, 464, 184, 1171, 1409...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, -1, -1, 0..."


Normalize data:

In [10]:
from sklearn.preprocessing import StandardScaler

data = {}
scalers = {}
for stat in stats:
    scalers[stat] = StandardScaler()
    for row in df[stat]:
        scalers[stat].partial_fit(np.asanyarray(row).reshape(-1, 1))
    data[stat] = [scalers[stat].transform(np.asanyarray(row).reshape(-1, 1)).reshape(-1) for row in df[stat]]

num_features = len(data)
print(f'# of features per timestep: {num_features}')

# of features per timestep: 7


Build dataset:

In [11]:
class LOLDataset(Dataset):
    def __init__(self, data, stats, label):
        
        self.data =[]
        for t in range(MAX_TIME_STEP):
            self.data.append([[data[stat][i][t] for stat in stats] for i in range(matches)])
        # print("shape:", np.asanyarray(self.data).shape)
        self.label=[i for i in label]
        
    
    def __getitem__(self, item):
        return torch.tensor([ [torch.scalar_tensor(i) for i in x[item]] for x in self.data]), torch.tensor(self.label[item])

    def __len__(self):
        return len(self.label)
            

## Neural Network Structure

### Recurrent Neural Network (RNN)

In [12]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN,self).__init__()
        self.hidden_size = 256
        
        self.rnn= nn.RNN(
            nonlinearity = 'relu',
            input_size = num_features,
            hidden_size = self.hidden_size,
            num_layers = 1,
            batch_first = True
        )

        self.out = nn.Linear(self.hidden_size, 2)
    
    def forward(self,x):
        r_out, hn = self.rnn(x, torch.zeros(1, len(x), self.hidden_size))
        out = self.out(r_out[:, -1, :])
        return out


#### Long Short-Term Memory (LSTM) Network

In [13]:
class LSTMRNN(nn.Module):
    def __init__(self):
        super(LSTMRNN, self).__init__()
        self.hidden_size = 256
        self.lstm = nn.LSTM(
            input_size = num_features,
            hidden_size = self.hidden_size,
            num_layers = 1,
            batch_first = True
        )
        self.out = nn.Linear(self.hidden_size, 2)
    
    def forward(self,x):
        h0 = torch.zeros(1, len(x), self.hidden_size)
        c0 = torch.zeros(1, len(x), self.hidden_size)
        r_out, (h_n, h_c) = self.lstm(x, (h0, c0))
        out = self.out(r_out[:, -1, :])
        return out

## Training

Split the dataset into Train:Held-Out:Test = 6:2:2, so we can early stop when held-out accuracy drops.

Using a batch size of 32 to load dataset for training.

In [14]:
BATCH_SIZE = 32

dataset = LOLDataset(data, stats, df["bResult"])
test_size = valid_size = int(0.2 * len(dataset))
train_size = len(dataset) - test_size - valid_size

trainDataset, validDataset, testDataset = random_split(
    dataset = dataset,
    lengths = [train_size, valid_size, test_size],
    generator = torch.Generator().manual_seed(0)
)

trainLoader = DataLoader(trainDataset, batch_size = BATCH_SIZE, shuffle=True)
validLoader = DataLoader(validDataset, batch_size = BATCH_SIZE)
testLoader = DataLoader(testDataset, batch_size = BATCH_SIZE)

In [15]:
def train(dataloader, model, loss_fn, optimizer, mute = False):
    size = len(dataloader.dataset)
    for batch, (x, y) in enumerate(dataloader):
        x, y = Variable(x), Variable(y)

        predict = model(x)
        loss = loss_fn(predict, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 30 == 0 and not mute:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [16]:
def test(dataloader, model, loss_fn, validation = False):
    model.eval()
    size = len(dataloader.dataset)

    correct = 0
    test_loss = 0
    with torch.no_grad():
        for step,(x,y) in enumerate(dataloader):
            x, y = Variable(x), Variable(y)
            predict = model(x)
            # print(predict)
            test_loss += loss_fn(predict, y).item()
            correct += (predict.argmax(1) == y).sum().item()
    
    print(f"{'Valid' if validation else 'Test'} Acc:{correct/size:>7f}, Avg Loss: {test_loss/size:>7f}")
    return correct/size

### Training RNN

In [17]:
# lr = 0.0005 acc=0.8386

MUTE = False
EPOCH = 100
LR = 0.0001

torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

model_RNN = RNN()
print(model_RNN)

optimizer = torch.optim.Adam(model_RNN.parameters(), lr = LR)
loss_func = nn.CrossEntropyLoss()

best_acc = 0
early_stopping = 0
early_stopping_threshold = 5

for epoch in range(1, EPOCH + 1):
    print(f"--------- Epoch #{epoch} ---------")
    train(trainLoader, model_RNN, loss_func, optimizer, mute = MUTE)
    valid_acc = test(validLoader, model_RNN, loss_func, validation = True)
    if valid_acc > best_acc :
        early_stopping = 0
        best_acc = valid_acc
        torch.save(model_RNN.state_dict(), "./model/RNN_"+str(MAX_TIME_STEP)+".pt")
    else :
        early_stopping += 1
        if early_stopping == early_stopping_threshold :
            print(f"Early stopped at epoch #{epoch} with best validation accuracy {best_acc*100:.2f}%.")
            break


RNN(
  (rnn): RNN(7, 256, batch_first=True)
  (out): Linear(in_features=256, out_features=2, bias=True)
)
--------- Epoch #1 ---------
loss: 0.662169  [    0/ 3832]
loss: 0.442477  [  960/ 3832]
loss: 0.354426  [ 1920/ 3832]
loss: 0.329362  [ 2880/ 3832]
Valid Acc:0.829154, Avg Loss: 0.011723
--------- Epoch #2 ---------
loss: 0.499428  [    0/ 3832]
loss: 0.395210  [  960/ 3832]
loss: 0.288157  [ 1920/ 3832]
loss: 0.320816  [ 2880/ 3832]
Valid Acc:0.831505, Avg Loss: 0.011421
--------- Epoch #3 ---------
loss: 0.221396  [    0/ 3832]
loss: 0.415795  [  960/ 3832]
loss: 0.357547  [ 1920/ 3832]
loss: 0.321554  [ 2880/ 3832]
Valid Acc:0.833856, Avg Loss: 0.011364
--------- Epoch #4 ---------
loss: 0.212827  [    0/ 3832]
loss: 0.257887  [  960/ 3832]
loss: 0.373306  [ 1920/ 3832]
loss: 0.361389  [ 2880/ 3832]
Valid Acc:0.829937, Avg Loss: 0.011272
--------- Epoch #5 ---------
loss: 0.274921  [    0/ 3832]
loss: 0.366087  [  960/ 3832]
loss: 0.228666  [ 1920/ 3832]
loss: 0.203982  [ 2880/

### Training LSTM RNN

In [18]:
model_LSTM = LSTMRNN()
print(model_LSTM)

LR = 0.0001
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

optimizer = torch.optim.Adam(model_LSTM.parameters(), lr = LR)
loss_func = nn.CrossEntropyLoss()

best_acc = 0
early_stopping = 0
early_stopping_threshold = 5

for epoch in range(1, EPOCH + 1):
    print(f"--------- Epoch #{epoch} ---------")
    train(trainLoader, model_LSTM, loss_func, optimizer, mute = MUTE)
    valid_acc = test(validLoader, model_LSTM, loss_func, validation = True)
    if valid_acc > best_acc :
        early_stopping = 0
        best_acc = valid_acc
        torch.save(model_LSTM.state_dict(), "./model/LSTM_"+str(MAX_TIME_STEP)+".pt")
    else :
        early_stopping += 1
        if early_stopping == early_stopping_threshold :
            print(f"Early stopped at epoch #{epoch} with best validation accuracy {best_acc * 100:.2f}%.")
            break

LSTMRNN(
  (lstm): LSTM(7, 256, batch_first=True)
  (out): Linear(in_features=256, out_features=2, bias=True)
)
--------- Epoch #1 ---------
loss: 0.726767  [    0/ 3832]
loss: 0.599416  [  960/ 3832]
loss: 0.468343  [ 1920/ 3832]
loss: 0.289526  [ 2880/ 3832]
Valid Acc:0.821317, Avg Loss: 0.012629
--------- Epoch #2 ---------
loss: 0.475134  [    0/ 3832]
loss: 0.462687  [  960/ 3832]
loss: 0.394772  [ 1920/ 3832]
loss: 0.311120  [ 2880/ 3832]
Valid Acc:0.829937, Avg Loss: 0.012108
--------- Epoch #3 ---------
loss: 0.438653  [    0/ 3832]
loss: 0.249605  [  960/ 3832]
loss: 0.227378  [ 1920/ 3832]
loss: 0.313090  [ 2880/ 3832]
Valid Acc:0.832288, Avg Loss: 0.011838
--------- Epoch #4 ---------
loss: 0.495733  [    0/ 3832]
loss: 0.592750  [  960/ 3832]
loss: 0.334342  [ 1920/ 3832]
loss: 0.351207  [ 2880/ 3832]
Valid Acc:0.837774, Avg Loss: 0.011661
--------- Epoch #5 ---------
loss: 0.475349  [    0/ 3832]
loss: 0.491948  [  960/ 3832]
loss: 0.253998  [ 1920/ 3832]
loss: 0.296170  [

## Test

Load best models for both RNN and LSTM and test for results:

In [19]:
print("RNN:")
model_RNN.load_state_dict(torch.load("./model/RNN_"+str(MAX_TIME_STEP)+".pt"))
acc_RNN = test(testLoader, model_RNN, loss_func)

print("LSTM RNN:")
model_LSTM.load_state_dict(torch.load("./model/LSTM_"+str(MAX_TIME_STEP)+".pt"))
acc_LSTM = test(testLoader, model_LSTM, loss_func)

print(f'RNN Accuracy = {acc_RNN*100:>.2f}% | LSTM Accuracy = {acc_LSTM*100:>.2f}%')

RNN:
Test Acc:0.844044, Avg Loss: 0.010316
LSTM RNN:
Test Acc:0.849530, Avg Loss: 0.010769
RNN Accuracy = 84.40% | LSTM Accuracy = 84.95%


Predict S11 World Championship Final: EDG vs DK

In [174]:
match_stats = {}

''' S11 EDG vs DK, match 3, DK(red) wins '''
match_stats['golddiff'] = [0,31,16,122,-71,325,170,367,463,918,1479,1181,689,813,890,1486,2779,2215,2543,2805,2887,2975,3477,3842,3361,3450,3318,3101,754,752]
match_stats['dragondiff'] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -2, -2, -2, -2, -2, -2, -3, -3, -3, -3, -3, -3, -4, -4]
match_stats['barondiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
match_stats['heralddiff'] = [0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
match_stats['towerdiff'] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2]
match_stats['inhibitordiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
match_stats['killdiff'] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -1, -1]


''' S11 EDG vs DK, match 4, EDG(blue) wins '''
# match_stats['golddiff'] = [0,-33,147,160,-124,-627,621,979,577,759,1136,1175,-39,554,2057,1839,2071,1771,2058,2515,2297,1408,1575,1459,1521,1525,2672,2314,2717,2417,]
# match_stats['dragondiff'] = [0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,3,3,3,3,3,4,4,4,4]
# match_stats['barondiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# match_stats['heralddiff'] = [0,0,0,0,0,0,0,0,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]
# match_stats['towerdiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1]
# match_stats['inhibitordiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# match_stats['killdiff'] = [0,0,0,0,0,0,1,1,1,1,1,1,-1,-1,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1]


''' S11 DK vs EDG, match 5, EDG(red) wins ''' 
# match_stats['golddiff'] = [0,-21,-108,135,125,451,233,133,-431,-810,-860,-1128,-794,-774,-797,-886,-401,-216,-321,35,-143,-55,-734,-1719,-1955,-1864,-2100,-487,-324,-3383]
# match_stats['dragondiff'] = [0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,0,0,0,-1,-1,-1,-1,-1,-2,-2,-2,-2,-2,-2,-3]
# match_stats['barondiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1]
# match_stats['heralddiff'] = [0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2]
# match_stats['towerdiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,-1]
# match_stats['inhibitordiff'] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
# match_stats['killdiff'] = [0,0,0,0,0,0,0,0,-1,-1,-2,-2,-3,-3,-3,-2,-2,-2,-2,-2,-2,-2,-5,-6,-6,-6,-6,-6,-6,-10]

for stat in stats:
    match_stats[stat] = scalers[stat].transform(np.asanyarray(match_stats[stat]).reshape(-1, 1)).reshape(-1)

x = np.asarray([[ [match_stats[stat][timestep] for stat in stats] for timestep in range(MAX_TIME_STEP) ]], dtype=np.float32)

print(x.shape)

def predict_one_match(model, x):
    model.eval()
    with torch.no_grad():
        x = torch.from_numpy(x)
        predict = model(x)
        # print(predict)
        winner = ['red', 'blue'][predict.argmax(1)]
        prob_red = math.exp(predict[0][0].item()) / (math.exp(predict[0][0].item()) + math.exp(predict[0][1].item()))
        prob_blue = math.exp(predict[0][1].item()) / (math.exp(predict[0][0].item()) + math.exp(predict[0][1].item()))
        print(f"model predicted winner: { winner }")
        print(f"red wins: {prob_red * 100 :.1f}% | blue wins: {prob_blue * 100:.1f}%")

print("RNN:")
predict_one_match(model_RNN, x)
print("LSTM RNN:")
predict_one_match(model_LSTM, x)

[30, 30, 30, 30, 30, 30, 30]
(1, 30, 7)
RNN:
model predicted winner: red
red wins: 68.8% | blue wins: 31.2%
LSTM RNN:
model predicted winner: red
red wins: 67.9% | blue wins: 32.1%
